In [ ]:
import collections
import heapq

import threading

class BoundedBlockingQueue(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.size = 0
        self.queue = collections.deque()
        self.condition = threading.Condition()
    def enqueue(self, element): # add element to queue if not full
        self.condition.acquire()
        while self.size >= self.capacity:
            self.condition.wait()
        self.queue += element
        self.size += 1
        self.condition.notifyAll()
        self.condition.release()
    def dequeue(self): # return last element in queue
        self.condition.acquire()
        while self.size == 0:
            self.condition.wait()
        result = self.queue.popleft()
        self.size -= 1
        self.condition.notifyAll()
        self.condition.release()
        return result
    def size(self):
        return self.size
    

In [ ]:
import collections
import heapq

'''
keyfreq - maps one key to one freq
freqkeys - maps one freq to many keys 
many keys are stored in OrderDict - double link has map
give a key, can find its frequency
give the frequency, we can find all keys with same frequency in order 
'''

class LFUCache:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.minfreq = None
        self.keyfreq = {}
        # double linked list + hash map or lru cache 
        self.freqkeys = defaultdict(OrderedDict)
    def get(self, key: int) -> int:
        if key not in self.keyfreq:
            return -1
        freq = self.keyfreq[key]
        val = self.freqkeys[freq][key]
        del self.freqkeys[freq][key]
        if not self.freqkeys[freq]:
            if freq == self.minfreq:
                self.minfreq += 1
            del self.freqkeys[freq]
        self.keyfreq[key] = freq+1
        self.freqkeys[freq+1][key] = val
        return val
    def put(self, key: int, value: int) -> None:
        if self.capacity <= 0: return
        if key in self.keyfreq:
            freq = self.keyfreq[key]
            self.freqkeys[freq][key] = value
            self.get(key)
            return
        if self.capacity == len(self.keyfreq):
            delkey, delval = self.freqkeys[self.minfreq].popitem(last=False)
            del self.keyfreq[delkey]
        self.keyfreq[key] = 1
        self.freqkeys[1][key] = value
        self.minfreq = 1
 

In [ ]:


class SnakeGame:
    def __init__(self, width: int, height: int, food: List[List[int]]):
        self.width = width
        self.height = height
        self.score = 0
        self.path = collections.deque([[0,0]])
        self.food_queue = collections.deque(food)
        if self.food_queue:
            self.food_now = self.food_queue.popleft()
    def move(self, direction: str) -> int:
        dirs = {'U':[1,0], 'D':[-1,0], 'L':[0,-1], 'R':[0,1]} 
        current_head = self.path[-1]
        new_head = [current_head[0]+dirs['direction'][0], current_head[1]+dirs['direction'][1]]
        if new_head[0] < 0 or new_head[1] < 0 or new_head[0] >= self.width or new_head[1] >= self.height:
            self.score = -1
        if new_head in self.path and new != self.path[0]: # tail moves out of the way 
            self.score = -1
        if new_head == self.foodnow:
            self.score += 1
            self.path.append(new)
            if self.food_queue:
                self.food_now = self.food_queue.popleft()
            else: # no food left
                self.food_now = [-1,-1]
        else: # did not eat food, move head and move tail
            self.path.append(new_head)
            self.path.popleft() # move tail 
        return self.score 
        


class HtmlParser(object):
    def getUrls(self, url):
        urls = ''
        return urls
class Solution:
    def crawl(self, startUrl, htmlParser): # single threaded
        def dfs(url, htmlParser):
            if url not in visited:
                visited.add(url)
                result.append(url)
            url_list = htmlParser.getUrls(url)
            for new_url in url_list:
                new_domain = new_url.split("http://")[1].split("/")[0]
                if new_domain == domain and new_url not in visited:
                    dfs(new_url, htmlParser)
        domain = startUrl.split("http://")[1].split("/")[0]
        visited = set()
        result = []
        dfs(startUrl, htmlParser)
        return result
from concurrent import futures
class Solution:
    # entries in queue are future objects
    # calling result on futures blocks until task is completed or rejected 
    def crawl(self, startUrl, htmlParser):
        domain = startUrl.split("http://")[1].split("/")[0]
        visited = set()
        visited.add(startUrl)
        n_max_threads = 16
        # with statment cleans up threadpool executor when done
        with futures.ThreadPoolExecutor(max_workers=n_max_threads) as executor:
            # append futures
            tasks = deque([executor.submit(htmlParser.getUrls, startUrl, timeout=0.15)])
            while tasks:
                for url in tasks.popleft().result(): # only grab results that are done
                    new_domain = url.split("http://")[1].split("/")[0]
                    if url not in visited and new_domain == domain:
                        visited.add(url)
                        tasks.append(executor.submit(htmlParser.getUrls, url, timeout=0.15))
        return list(visited)

 





# dbx tagged organized

class PhoneDirectory:
    def __init__(self, maxNumbers):
        self.nextAvailable = 0
        self.maxSize = maxNumbers
        self.released = set() 
    def get(self):
        if self.nextAvailable < self.maxSize:
            new_num = self.nextAvailable
            self.nextAvailable += 1
            return new_num
        else:
            if self.released:
                new_num = self.released.pop()
                return new_num
            else:
                return -1
    def check(self, number):
        if number >= self.nextAvailable and number <= self.maxSize:
            return True
        if number < self.nextAvailable and number in self.released:
            return True
        return False 
    def release(self, number):
        if number < self.nextAvailable:
            self.released.add(number)        
        
        

class Leaderboard:
    def __init__(self):
        self.scores = collections.defaultdict(int)
    def addScore(self, playerId, score):
        self.scores[playerId] += score
    def reset(self, playerId):
        self.scores[playerId] = 0
    def top(self, K): # T: O(nlogk)
        heap = []
        for score in self.scores.values():
            heappush(heap, score)
            if len(heap) > K:
                heappop(heap)        
        return sum(heap)

class HitCounter:
    def __init__(self):
        self.counter = {}
        for i in range(0, 300):
            self.counter[i] = [0,i+1]
        return        
    def hit(self, timestamp):
        i = int((timestamp - 1)%300)
        if self.counter[i][1] == timestamp:
            self.counter[i][0] += 1
        else: # existing timestamp is too old 
            self.counter[i][0] = 1            
            self.counter[i][1] = timestamp
    def getHits(self, timestamp):
        n_hits = 0
        for key, value in self.counter.items():
            count, dt = value[0], value[1]
            if timestamp - dt < 300: # recent data only 
                n_hits += count
        return n_hits
        


# dbx tagged not organized

            
    


In [ ]:
import collections
from concurrent import futures


In [ ]:

class Sea(object):
    def hasShips(self, topRight, bottomLeft):
class Point(object):
    def __init__(self, x: int, y: int):
        self.x = x
        self.y = y
class Solution: # (object)
    def countShips(self, sea, topRight, bottomLeft):
        def dfs(bottomLeft, topRight):
            if bottomLeft.x > topRight.x or bottomLeft.y > topRight.y:
                return 0
            if topRight.x == bottomLeft.x and topRight.y == bottomLeft.y:     
                return int(sea.hasShips(topRight, bottomLeft)) 
            else:
                if not sea.hasShips(topRight, bottomLeft): return 0
                x0, y0 = bottomLeft.x, bottomLeft.y
                x1, y1 = topRight.x, topRight.y
                x, y = (x0+x1)//2, (y0+y1)//2
                q1 = dfs(bottomLeft, Point(x, y))
                q2 = dfs(Point(x+1, y+1), topRight)
                q3 = dfs(Point(x0, y+1), Point(x, y1))
                q4 = dfs(Point(x+1, y0), Point(x1, y))
                return q1+q2+q3+q4
        return dfs(bottomLeft, topRight)           
        
    

In [ ]:
class RandomizedSet:
    def __init__(self):
        self.cache = {}
        self.nums = []
        self.length = 0
    def insert(self, val):
        if val in self.cache:
            return False
        else:
            self.nums.append(val)
            self.cache[val] = self.length
            self.length += 1
            return True
    def remove(self, val):
        if val not in self.cache:
            return False
        else: # move item to remove to end and pop
            index = self.cache[val]
            last = self.nums[self.length-1]
            self.nums[index] = last
            self.cache[last] = index
            del self.cache[val]
            self.nums.pop()
            self.length -= 1
            return True
    def getRandom(self):
        index = random.randint(0,self.length-1)
        return self.nums[index]
    

In [ ]:
class MinStack:
    def __init__(self):
        self.stack = []
    def push(self, x):
        new_min = min(self.getMin(), x)
        self.stack.append((x, new_min)) 
    def pop(self):
        self.stack.pop()
    def top(self):
        if self.stack:
            return self.stack[-1][0]
    def getMin(self):
        if self.stack:
            return self.stack[-1][1]
        else:
            return float('inf')
        
        
min_stack = MinStack()
min_stack.push(-2)
min_stack.push(0)
min_stack.push(-3)
min_stack.getMin()
min_stack.pop()
min_stack.top()
min_stack.getMin()
print(min_stack)
#print(temp1.maximumProduct(nums))
#print(nums)


In [ ]:
class Solution:
    def candyCrush(self, board: List[List[int]]) -> List[List[int]]:
        if not board:
            return []
        ny, nx = len (board), len (board[0])        
        self.crush_set = set()
        def look_for_matches():            
            for j in range (0, ny):
                for i in range (0, nx):                    
                    if j >= 2 and board[j][i] != 0 and board[j][i] == board[j-1][i] == board[j-2][i]:
                        self.crush_set.update([(j,i), (j-1,i), (j-2,i)])
                    if i >= 2 and  board[j][i] !=0 and board[j][i] == board[j][i-1] == board[j][i-2]:
                        self.crush_set.update([(j,i), (j,i-1), (j,i-2)])                    
            return
        def crush_candy(): 
            for j,i in self.crush_set:
                board[j][i] = 0                
            return            
        def drop():
            for i in range (0, nx):                
                stack = []
                # iterate over rows
                for j in range (ny-1, -1, -1):
                    if board[j][i] != 0:
                        stack.append(board[j][i])        
                stack += [0]*(ny-len(stack)) 
                j = 0
                while stack:
                    board[j][i] = stack.pop()
                    j += 1
            return
        look_for_matches()
        while self.crush_set:
            crush_candy()
            drop()
            self.crush_set = set() # clear matches
            look_for_matches()
        return board

col -nx
row - ny


In [ ]:

class UndergroundSystem:
    def __init__(self):
        self.checkInDict = collections.defaultdict(list)
        self.travelTimes = collections.defaultdict(list)        
    def checkIn(self, id, stationName, t):
        if id in self.checkInDict:
            print('ERROR - user cannot check in to to places at once')
        else:
            self.checkInDict[id] = [t, stationName]        
    def checkOut(self, id, stationName, t):
        if id not in self.checkInDict:
            print('ERROR - user cannot check out before checking in')
        else: 
            tStart, startStation = self.checkInDict[id]
            deltaTime = t - tStart
            stationPair = startStation+'-'+stationName
            self.travelTimes[stationPair].append(deltaTime)
            del self.checkInDict[id]        
    def getAverageTime(self, startStation, endStation):
        stationPair = startStation+'-'+endStation
        if stationPair not in self.travelTimes:
            print('ERROR - station pair not present')
        else:
            times = self.travelTimes[stationPair]
            avgTime = float(sum(times)/len(times))
            return avgTime
        

In [ ]:
import collections
from collections import deque
from collections import defaultdict
import heapq
import random

class BrowserHistory:
    def __init__(self, homepage):
        self.history = [homepage]
        self.index = 0 
        self.length = 0
    def visit(self, url): # append new or retrieve existing
        self.index += 1
        if self.index == len(self.history):
            self.history.append(url)
        else:
            self.history[self.index] = url
        self.length = self.index
    def back(self, steps):
        self.index = max(0, self.index-steps)
        return self.history[self.index]
    def forward(self, steps):
        self.index = min(self.length, self.index+steps)
        return self.history[self.index]
        
        
        
browserHistory = BrowserHistory()

#["BrowserHistory","visit","visit","visit","back","back","forward","visit","forward","back","back"]
#[["leetcode.com"],["google.com"],["facebook.com"],["youtube.com"],[1],[1],[1],["linkedin.com"],[2],[2],[7]]
        

# Your BrowserHistory object will be instantiated and called as such:
# obj = BrowserHistory(homepage)
# obj.visit(url)
# param_2 = obj.back(steps)
# param_3 = obj.forward(steps)    
    

In [ ]:
    def __init__(self, nums):
        self.cache = collections.defaultdict(list)
        for index,n in enumerate(nums):
            self.cache[n].append(index)
        #print(self.cache)
        
    def pick(self, target):
        #print(self.cache)
        if len(self.cache[target]) == 1:
            return self.cache[target][0]
        else:
            #rand_int = random.randrange(len_temp)
            #return self.cache[target][rand_int]
            return random.choice(self.cache[target])
